In [2]:
import pandas as pd

def add_biomarkers_as_features(compiled_metadata, data):
    compiled_metadata = compiled_metadata.set_index("Public Sample ID")
    compiled_metadata = compiled_metadata.sort_index()
    compiled_metadata = compiled_metadata.dropna(axis = 0, how = 'all')
    compiled_metadata = compiled_metadata[(compiled_metadata.Condition != 'Golden West CSF Batch QC')]
    compiled_metadata = compiled_metadata[compiled_metadata.Condition != 'Lumbar CSF Pool Batch Ref']
    compiled_metadata = compiled_metadata[['Condition', 'Ttau (pg/ml)', 'AB42 (pg/ml)']]
    dict_AB42_ratio = {}
    for i, j in zip(compiled_metadata.index, compiled_metadata['AB42 (pg/ml)']):
        dict_AB42_ratio[i]= j
        dict_tau = {}
    for i, j in zip(compiled_metadata.index, compiled_metadata['Ttau (pg/ml)']):
        dict_tau[i]= j
    data['AB42/AB40'] = data['assay_ID'].map(dict_AB42_ratio)
    data['Ttau (pg/ml)'] = data['assay_ID'].map(dict_tau)
    return data

In [3]:
data_full = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/split_data/full_data_short.csv')
metadata = pd.read_csv('/Users/mariamb/Desktop/BrainPower/brainpower/data/metadata.csv')
data_full = add_biomarkers_as_features(metadata, data_full)
data_full = data_full.drop(columns='assay_ID')
data_full.to_csv("data_with_biomarkers.csv", sep='\t', index=False)

In [6]:
data_full

,group,KV37,LV469,LV861,LVX54,LV746,LV218,LV316,LV312,LV310,...,TEN1,PCDAD,ITM2B,ADSV,A0A1W2PRN1,APOF,DCBD2,LMF2,AB42/AB40,Ttau (pg/ml)
0,AD_MCI,28.898373,24.125336,22.676771,20.327832,25.570557,24.797337,23.543465,22.555490,22.783560,...,17.856492,18.453124,NaN,20.879716,24.781034,NaN,NaN,NaN,918.0,844.0
1,AD_MCI,28.915048,22.824361,23.197649,19.990306,25.316286,23.571908,24.051694,22.300534,23.046184,...,16.816413,17.987179,NaN,NaN,NaN,19.805173,21.125776,NaN,551.0,1161.0
2,AD_MCI,28.274292,24.358802,24.066243,23.589011,26.241142,26.079288,24.586944,24.266694,23.910209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,846.0,220.0
3,AD_MCI,29.193195,22.187373,23.550432,20.329372,25.789922,24.700275,23.881087,22.998723,22.430900,...,17.568634,16.772132,16.934531,18.446332,24.750659,NaN,NaN,NaN,470.0,576.0
4,AD_MCI,28.212282,22.520379,24.504620,20.681650,26.287648,25.020801,23.176368,23.293055,22.188477,...,18.116146,18.547994,18.692276,NaN,NaN,20.264877,21.677769,16.254727,479.0,771.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
275,PD_MCI_LBD,28.131446,22.064598,21.473568,20.811029,24.925192,23.815794,22.649795,22.365696,22.055298,...,NaN,19.603221,17.691827,19.303921,NaN,22.159498,NaN,13.719437,1682.0,291.0
276,PD_MCI_LBD,28.156999,24.163694,23.342793,15.607011,25.250506,24.746199,24.264510,21.975955,22.122501,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,345.0,178.0
277,PD_MCI_LBD,29.919698,23.727972,24.024882,16.081469,26.457936,25.880806,24.292395,23.740719,23.743020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,576.0,285.0
278,PD_MCI_LBD,29.254995,22.621331,23.667065,23.309039,26.553185,24.716706,23.063840,22.399448,22.182852,...,NaN,15.844641,15.265794,20.190391,NaN,20.998117,NaN,16.722097,296.0,207.0


If I want to prep the raw data to be in the format to be ready for the feature selection and ML model train/test process:

In [ ]:
df = handle_scale_and_nan(data_full)
data_dev, data_test = split_cats_by_tolerance(df,0.01,randomstate=98281)
data_dev = over_under(data_dev,cat_in_excess='Healthy',target='group',randomstate=np.random.randint(0,4294967295))